In [ ]:
import os

BASE_DIR = '/home/rithomas'
# folder containing MIDI and/or MusicXML files. can have child folders.
RAW_DATA = os.path.join(BASE_DIR, 'data', 'clean_midi')
# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD = os.path.join(BASE_DIR, 'cache', 'polyphonyRNN', 'data', 'notesequences.tfrecord')
# Model config must be one of ['basic_improv', 'attention_improv', 'chord_pitches_improv']
POLYPHONY_RNN_CONFIG = 'chord_pitches_improv'
# Model directory for storing weights and tensorboard stuff
POLYPHONY_RNN_DIR = os.path.join(BASE_DIR, 'cache', 'polyphonyRNN', 'model')
# Dataset of Sequence Examples for the ImprovRNN Model
POLYPHONY_RNN_DATASET = os.path.join(BASE_DIR, 'cache', 'polyphonyRNN')
# Training sequence examples
TRAINING_EXAMPLES = os.path.join(POLYPHONY_RNN_DATASET, 'training_lead_sheets.tfrecord')
# Dir for testing outputs
OUTPUTS_PATH = os.path.join(POLYPHONY_RNN_DATASET, 'outputs')
# Final bundled model
BUNDLE_DIR = os.path.join(POLYPHONY_RNN_DIR, POLYPHONY_RNN_CONFIG + '.mag')

if not os.path.exists(SEQUENCES_TFRECORD):
    !python3 "magenta/scripts/convert_dir_to_note_sequences.py" \
      --input_dir=$RAW_DATA \
      --output_file=$SEQUENCES_TFRECORD \
      --recursive

In [ ]:
polyphony_rnn_create_dataset \
--input=/tmp/notesequences.tfrecord \
--output_dir=/tmp/polyphony_rnn/sequence_examples \
--eval_ratio=0.10


In [ ]:
polyphony_rnn_train \
--run_dir=/tmp/polyphony_rnn/logdir/run1 \
--sequence_example_file=/tmp/polyphony_rnn/sequence_examples/training_poly_tracks.tfrecord \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--num_training_steps=20000

In [ ]:
polyphony_rnn_generate \
--run_dir=/tmp/polyphony_rnn/logdir/run1 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--output_dir=/tmp/polyphony_rnn/generated \
--num_outputs=10 \
--num_steps=128 \
--primer_pitches="[67,64,60]" \
--condition_on_primer=true \
--inject_primer_during_generation=false

In [ ]:
polyphony_rnn_generate \
--run_dir=/tmp/polyphony_rnn/logdir/run1 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--bundle_file=/tmp/polyphony_rnn.mag \
--save_generator_bundle